In [1]:
%matplotlib inline

import cv2 # The OpenCV library; install using `pip install opencv-contrib-python`
import numpy as np # Helpful when working with arrays; install using `pip install numpy`
from matplotlib import pyplot as plt # Good for graphing; install using `pip install matplotlib`
from matplotlib import image as image

In [8]:
import cv2
import numpy as np

# Video and background subtraction initialization
video = cv2.VideoCapture("Images/traffic.mp4")
sub_type = 'MOG2'
if sub_type == "MOG2":
    backSub = cv2.createBackgroundSubtractorMOG2(varThreshold=16, detectShadows=True)
    backSub.setShadowThreshold(0.5)
else:
    backSub = cv2.createBackgroundSubtractorKNN(dist2Threshold=1000, detectShadows=True)

# Parameters for morphological operations
kernel = np.ones((9,9), dtype=np.uint8)

# Kalman filter setup
kalman_filters = []
tracked_objects = []

# Helper function to initialize a Kalman filter
def create_kalman_filter():
    kf = cv2.KalmanFilter(4, 2)
    kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
    kf.processNoiseCov = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 5, 0], [0, 0, 0, 5]], np.float32) * 0.03
    return kf

# Main loop
while True:
    ret, frame = video.read()
    if not ret:
        break

    # Background subtraction and thresholding
    fg_mask = backSub.apply(frame)
    _, thresh = cv2.threshold(fg_mask, 0, 255, cv2.THRESH_BINARY)
    motion_mask = cv2.medianBlur(thresh, 3)
    
    # Morphological operations
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_CLOSE, kernel, iterations=1)

    # Find contours in the motion mask
    contours, _ = cv2.findContours(motion_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    detected_centers = []
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Filter small contours
            x, y, w, h = cv2.boundingRect(contour)
            center = np.array([[np.float32(x + w / 2)], [np.float32(y + h / 2)]])
            detected_centers.append(center)

    # Update Kalman filters and track objects
    new_tracked_objects = []
    for center in detected_centers:
        matched = False
        for i, kf in enumerate(kalman_filters):
            # Prediction step
            predicted = kf.predict()
            predicted_center = np.array([predicted[0], predicted[1]])
            distance = np.linalg.norm(center - predicted_center)
            
            if distance < 50:  # Match based on distance threshold
                # Update step with new measurement
                kf.correct(center)
                new_tracked_objects.append(kf)
                matched = True
                break

        # If no match found, initialize a new Kalman filter for the detected object
        if not matched:
            kf = create_kalman_filter()
            kf.statePre = np.array([center[0, 0], center[1, 0], 0, 0], dtype=np.float32)
            kf.correct(center)
            kalman_filters.append(kf)
            new_tracked_objects.append(kf)

    # Remove old kalman filters if not matched (e.g., vehicles leaving frame)
    kalman_filters = new_tracked_objects

    # Draw tracking results
    for kf in kalman_filters:
        predicted = kf.predict()
        x, y = int(predicted[0]), int(predicted[1])
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

    # Display frames
    cv2.imshow('Frame', frame)
    cv2.imshow('Motion Mask', motion_mask)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
video.release()
cv2.destroyAllWindows()
